In [3]:
!python neat_test.py


[***]	Fitness: [720.91]


In [4]:
from neat_src.ann import getNodeOrder

import numpy as np

# Create a more complex network with multiple hidden nodes and connections
nodeG = np.array([
    [5, 6, 7, 8, 9, 10],     # Node IDs: 1 input, 1 bias, 1 output, 3 hidden
    [1, 4, 2, 3, 3, 3],      # Node types: 5=input, 6=bias, 7=output, rest hidden
    [1, 1, 1, 1, 1, 1]       # Activation functions (1 = linear)
])

connG = np.array([
    [1, 2, 3, 4, 5, 6],      # Innovation numbers
    [5, 6, 8, 9, 8, 10],     # Source nodes
    [8, 8, 9, 7, 10, 7],     # Destination nodes
    [0.5, 0.3, 0.2, 0.4, 0.6, 0.1],  # Weights
    [1, 1, 1, 1, 1, 1]       # All enabled
])

# Test the function
Q, wMat = getNodeOrder(nodeG, connG)
print("Node order Q:", Q)
print("\nWeight Matrix:")
print(wMat)

Node order Q: [0 1 3 4 5 2]

Weight Matrix:
[[0.  0.  0.  0.5 0.  0. ]
 [0.  0.  0.  0.3 0.  0. ]
 [0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  1.  1. ]
 [0.  0.  0.4 0.  0.  0. ]
 [0.  0.  0.1 0.  0.  0. ]]


In [6]:
from neat_src.ann import getLayer 

getLayer(wMat)

array([0., 0., 3., 1., 2., 2.])

In [1]:
from neat_src.ind import Ind 

import numpy as np

# Create a more complex network with multiple hidden nodes and connections
nodeG = np.array([
    [5, 6, 7, 8, 9, 10],     # Node IDs: 1 input, 1 bias, 1 output, 3 hidden
    [1, 4, 2, 3, 3, 3],      # Node types: 5=input, 6=bias, 7=output, rest hidden
    [1, 1, 1, 1, 1, 1]       # Activation functions (1 = linear)
])

connG = np.array([
    [1, 2, 3, 4, 5, 6],      # Innovation numbers
    [5, 6, 8, 9, 8, 10],     # Source nodes
    [8, 8, 9, 7, 10, 7],     # Destination nodes
    [0.5, 0.3, 0.2, 0.4, 0.6, 0.1],  # Weights
    [1, 1, 1, 1, 1, 1]       # All enabled
])

individual = Ind(connG, nodeG)

In [5]:
# individual.mutAddConn(connG, nodeG, None, 1, 0.5) # takes forever -- likely into infinite loop ... 

In [4]:
from neat_src.ann import getNodeOrder, getLayer
newConnId = connG[0, -1] + 1

nIns = len(nodeG[0,nodeG[1,:] == 1]) + len(nodeG[0,nodeG[1,:] == 4])
nOuts = len(nodeG[0,nodeG[1,:] == 2])
print("Sort node with topological sort")
order, wMat = getNodeOrder(nodeG, connG)   # Topological Sort of Network
print("-- sorting complete")
hMat = wMat[nIns:-nOuts,nIns:-nOuts]


print("Get hidden layer number")
hLay = getLayer(hMat)+1 # hidden node layer number
print("-- hidden layer number complete")

print("Assign layer number to each node")
# To avoid recurrent connections nodes are sorted into layers, and connections are only allowed from lower to higher layers
if len(hLay) > 0:
    lastLayer = max(hLay)+1 # output node layer number
else:
    lastLayer = 1 # output node layer number 
    
L = np.r_[np.zeros(nIns), hLay, np.full((nOuts),lastLayer) ] # Assign layer number to each node

nodeKey = np.c_[nodeG[0,order], L] # node key: node id and layer number (ordered by layer number)

sources = np.random.permutation(len(nodeKey)) # node index permutation
for src in sources:
    srcLayer = nodeKey[src,1] # take source node according to index
    dest = np.where(nodeKey[:,1] > srcLayer)[0] # pick all nodes in higher layers
    
    # Finding already existing connections:
    #   ) take all connection genes with this source (connG[1,:])
    #   ) take the destination of those genes (connG[2,:])
    #   ) convert to nodeKey index (Gotta be a better numpy way...)   
    srcIndx = np.where(connG[1,:]==nodeKey[src,0])[0]
    exist = connG[2,srcIndx]
    existKey = []
    for iExist in exist:
        existKey.append(np.where(nodeKey[:,0]==iExist)[0])
        dest = np.setdiff1d(dest,existKey) # Remove existing connections

Sort node with topological sort
-- sorting complete
Get hidden layer number
-- hidden layer number complete
Assign layer number to each node


In [5]:
existKey

[]

In [11]:
connG

array([[ 1. ,  2. ,  3. ,  4. ,  5. ,  6. ],
       [ 5. ,  6. ,  8. ,  9. ,  8. , 10. ],
       [ 8. ,  8. ,  9. ,  7. , 10. ,  7. ],
       [ 0.5,  0.3,  0.2,  0.4,  0.6,  0.1],
       [ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ]])

In [9]:
hMat

array([[0., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [13]:
wMat[np.isnan(wMat)] = 0  
wMat[wMat!=0]=1
nNode = np.shape(wMat)[0]
layer = np.zeros((nNode))

layer

array([0., 0., 0.])

In [16]:
wMat.T

array([[0., 0., 1.],
       [0., 0., 0.],
       [0., 1., 0.]])

In [22]:
wMat = hMat 

print("Get layer number for each node")
wMat[np.isnan(wMat)] = 0  
wMat[wMat!=0]=1
nNode = np.shape(wMat)[0]
layer = np.zeros((nNode))

while True: 
    prevOrder = np.copy(layer)
    srcLayer = (wMat.T * layer[np.newaxis,:])
    layer = np.max(srcLayer,axis=1)+1
    print("srcLayer: ", srcLayer)
    print("Layer: ", layer)
    
    if np.array_equal(prevOrder,layer):
        break
    

Get layer number for each node
srcLayer:  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Layer:  [1. 1. 1.]
srcLayer:  [[0. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]]
Layer:  [2. 1. 2.]
srcLayer:  [[0. 0. 2.]
 [0. 0. 0.]
 [0. 1. 0.]]
Layer:  [3. 1. 2.]
srcLayer:  [[0. 0. 2.]
 [0. 0. 0.]
 [0. 1. 0.]]
Layer:  [3. 1. 2.]


In [17]:
layer[np.newaxis, :]

array([[0., 0., 0.]])

In [20]:
srcLayer = (wMat.T * layer[np.newaxis, :])
np.max(srcLayer, axis=1)

array([0., 0., 0.])